In [456]:
DATA = """
########################
#f.D.E.e.C.b.A.@.a.B.c.#
######################.#
#d.....................#
########################
""".strip()

In [457]:
import six
import numpy as np

In [460]:
class Map(object):
    
    def __init__(self, layout):
        """
        Initialises the map given its layout.
        
        :param layout: str
        """
        self.layout = [[c for c in row] for row in layout.splitlines()]
        self.width, self.height = len(self.layout[0]), len(self.layout)
        self.start = self.locationOf('@')    
        self.numberOfKeys = sum(int(c.isalpha() and c.lower() == c) for c in str(self.layout))
        
    def locationOf(self, z):
        """
        Finds the location of the item with the given value.
        
        :param z: str
        :return: int, int
        """
        for y, row in enumerate(self.layout):
            if z in row:
                return row.index(z), y   
            
    def read(self, x, y):
        """
        Reads the item found at the given coordinates, if any.
        
        :param x: int
        :param y: int
        :return: str
        """
        try: return self.layout[y][x]
        except Exception: return None
        
    def getNeighbours(self, x, y):
        """
        Yields the neighbouring locations, aside from walls.
        
        :param x: int
        :param y: int
        :yield: int, int
        """
        directions = [(1, 0), (-1, 0), (0, 1), (0, -1)]
        for dx, dy in directions:
            nx, ny = x + dx, y + dy
            v = self.read(nx, ny)
            if not v: continue
            elif v == '#': continue
            else: yield (nx, ny)
    
    def isDoor(self, v):
        """
        Wheter the given item is a door.
        
        :param v: str
        :return: bool
        """
        return v and v.isalpha() and v.upper() == v
    
    def isKey(self, v):
        """
        Wheter the given item is a key.
        
        :param v: str
        :return: bool
        """
        return v and v.isalpha() and v.lower() == v
    
    def isWalkable(self, v):
        """
        Wheter the given item is a walkable location.
        
        :param v: str
        :return: bool
        """
        return v in ('@', '.')
                
    def solve(self):
        
        """
        Finds the minimum number of steps required to collect all keys.
        
        :return: int
        """
        
        # The approach taken to solve this task employs BFS, that is,
        # from the starting state, one step is taken in every direction
        # and all new states are recorded; then a step is taken in every
        # direction from all the states found at the previous iteration.
        # This guarantees that the first time that all keys are held at
        # a state, that state will have been reached optimally (in terms
        # of number of steps taken).
        
        # To improve performance, a lookup is kept for all visited states,
        # mapping locations to all distinct sets of keys held at that
        # state every time it was reached. This way, if a certain location
        # is reached and the keys held (or a superset of it) once it is
        # reached were previously held at that location, then it is
        # guaranteed that the searching process has already gone down that
        # road.
        
        # While a solution for the input puzzle can be retrieved within
        # ~5 minutes, there certainly remain better approaches to this
        # task.
        
        
        
        # This set will contain all states which need inspecting; a state
        # is a tuple of the location coordinates, the keys collected until
        # that state and the number of steps taken;
        
        states = {(self.start, (), 0),}
        
        # This map contains all visited states, with coordinates as keys and
        # a tuple of keys held at that coordinates and number of steps taken
        # until that state as value;
        
        visited = {}
        
        while states:   
            
            for state in set(states):
                
                (x, y), keys, steps = state
                
                # Remove the current state from the set of those which
                # still need inspecting;
                states.remove(state)
                
                # Add the location of the current state and its associated values to
                # the visited lookup;
                if (x, y) not in visited: visited[(x, y)] = []
                visited[(x, y)].append(keys)              
                
                for neighbour in self.getNeighbours(x, y):
                    # For every neighbouring location of the location of the current state;
                    newKeys = set(keys)
                    v = self.read(*neighbour)
                    
                    if self.isKey(v) and v not in newKeys:
                        # If it's the last key, return the total number of steps taken
                        # until the current state, otherwise add it to the set of keys
                        # for the current state;
                        if len(newKeys) + 1 == self.numberOfKeys: return steps + 1
                        else: newKeys.add(v)
                    elif self.isDoor(v) and v.lower() not in newKeys:
                        # If it's a door and its key was not found until the current
                        # state, disregard the state;
                        continue
         
                    if any(newKeys.issubset(k) for k in visited.get(neighbour, [])):
                        # If the current state was previously reached with at least
                        # all the keys obtain until the current state, disregard it;
                        continue
                    else:
                        # Otherwise add the neighbour to the set of states to be
                        # inspected at the next iteration;
                        states.add((neighbour, tuple(sorted(newKeys)), steps+1))
                

In [461]:
m = Map(DATA)
m.solve()

86